<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# REST-APIs mit Python: Einführung in das requests Modul

Das requests-Modul ist eine der beliebtesten Python-Bibliotheken für die Arbeit mit Web-APIs. Es bietet eine einfache Möglichkeit, HTTP-Requests zu senden und API-Daten abzurufen, zu bearbeiten oder zu löschen. Kurz gesagt: requests vereinfacht den Zugriff auf APIs, indem es das Senden von HTTP-Methoden wie GET, POST, PUT und DELETE erleichtert.
<br>
<br>
Im laufe dieses Kapitels nutzen wir die Open-Meteo API. Es handelt sich einfach nur um eine kostenlose API, um aktuelle Wetterdaten für verschiedene Städte abzurufen und zu analysieren.
<br>
<br>
Die Basis-URL der API ist:

```
https://api.open-meteo.com/
```

Jeder Endpunkt liefert spezifische Daten, z. B.:
- `/v1/forecast?latitude={LAT}&longitude={LON}&current_weather=true`:
  - Liefert aktuelle Wetterdaten für die angegebene Breiten- und Längengrade.
- `v1/forecast?latitude={LAT}&longitude={LON}&daily=temperature_2m_max,temperature_2m_min&timezone={ZEITZONE}`:
  - Liefert eine mehrtägige Wettervorhersage mit Höchst- und Tiefsttemperaturen.
- `v1/forecast?latitude={LAT}&longitude={LON}&hourly=temperature_2m,windspeed_10m&timezone={ZEITZONE}`:
  - Liefert stündliche Wetterdaten für Temperatur und Windgeschwindigkeit.
- `v1/forecast?latitude={LAT}&longitude={LON}&daily=uv_index_max,shortwave_radiation_sum&timezone={ZEITZONE}`:
  - Gibt den maximalen UV-Index sowie die tägliche Sonnenstrahlung zurück.
- `v1/forecast?latitude={LAT1},{LAT2}&longitude={LON1},{LON2}&current_weather=true`:
  - Man kann das Wetter für mehrere Standorte gleichzeitig in einem einzigen API-Request anfragen.

Als erstes müssen wir das `requests` Modul installieren:

```
pip3 install requests
```
## Einführungsbeispiel

Wir wollen als Einführungsbeispiel die aktuellen Wetterdaten von Berlin abrufen.

```python
import requests
import json

LATITUDE = 52.52  
LONGITUDE = 13.41  
BASE_URL = "https://api.open-meteo.com/"
ENDPOINT_1 = f"v1/forecast?latitude={LATITUDE}&longitude={LONGITUDE}&current_weather=true"

# Get-Request:
response = requests.get(BASE_URL + ENDPOINT_1)

# HTTP-Statuscode prüfen:
if response.status_code == 200:
    # "response.json()" wandelt die API-Antwort (die als JSON-String kommt) in ein Python-Dictionary oder eine Liste um
    # "json.dumps()" wandelt das Dictionary/Liste zurück in einen formatierten JSON-String.
    print(json.dumps(response.json(), indent=4, sort_keys=True))
else:
    print("Fehler beim Abrufen der Wetterdaten")
```

Der Response-Body sieht dann in etwa so aus:
```
{
    "current_weather": {
        "interval": 900,
        "is_day": 0,
        "temperature": 7.2,
        "time": "2025-01-30T16:30",
        "weathercode": 3,
        "winddirection": 194,
        "windspeed": 5.9
    },
    "current_weather_units": {
        "interval": "seconds",
        "is_day": "",
        "temperature": "\u00b0C",
        "time": "iso8601",
        "weathercode": "wmo code",
        "winddirection": "\u00b0",
        "windspeed": "km/h"
    },
    "elevation": 38.0,
    "generationtime_ms": 0.039458274841308594,
    "latitude": 52.52,
    "longitude": 13.419998,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "utc_offset_seconds": 0
}
```

Betrachten wir die gelieferten Daten genauer:

| Schlüssel                  | Wert                           | Erklärung                                                                                    |
| -------------------------- | ------------------------------ | -------------------------------------------------------------------------------------------- |
| "interval": 900            | 900 Sekunden                   | Aktualisierungsintervall der Wetterdaten (alle 15 Minuten).                                  |
| "is_day": 0                | 0 (Nacht)                      | Gibt an, ob es gerade Tag (1) oder Nacht (0) ist.                                            |
| "temperature": 7.2         | 7.2°C                          | Aktuelle Temperatur in Grad Celsius.                                                         |
| "time": "2025-01-30T16:30" | 30. Januar 2025, 16:30 Uhr UTC | Zeitpunkt der Wettermessung. (UTC-Zeitformat)                                                |
| "weathercode": 3           | 3 (Bewölkt)                    | WMO-Wettercode (0 = Klar, 1 = Teilweise bewölkt, 3 = Bewölkt, 45 = Nebel, 61 = Regen, etc.). |
| "winddirection": 194       | 194° (Süd-Südwest)             | Windrichtung in Grad (0° = Norden, 90° = Osten, 180° = Süden, 270° = Westen).                |
| "windspeed": 5.9           | 5.9 km/h                       | Aktuelle Windgeschwindigkeit in km/h.                                                        |
| "current_weather_units"    | Siehe Response                 | Zeigt an, welche Einheiten die Daten unter "current_weather" besitzen.                       |
| "elevation"                | 38.0                           | Die Höhe über dem Meeresspiegel für den angegebenen Standort in Metern.                      |
| "generationtime_ms"        | 0.039458274841308594           | Die Zeit, die die API benötigt hat, um die Daten zu berechnen, in Millisekunden.             |
| "latitude"                 | 52.52                          | Breitengrad                                                                                  |
| "longitude"                | 13.419998                      | Längengrad                                                                                   |
| "timezone"                 | "GMT"                          | Die Zeitzone, für die die Daten generiert wurden (hier Greenwich Mean Time (GMT)).           |
| "timezone_abbreviation":   | "GMT"                          | Die Abkürzung der Zeitzone.                                                                  |
| "utc_offset_seconds"       | 0                              | Der Zeitunterschied zu UTC in Sekunden (0 bedeutet keine Verschiebung, also UTC/GMT-Zeit).   |

Nun können wir viele interessante Informationen aus diesen Daten Gewinnen. Betrachten wir dazu einfach einige Punkte.

**1. Temperatur abrufen und ausgeben**

Wir erstellen ein kleines Programm, mit dem wir die aktuelle Temperatur für Berlin ausgeben können:

```python
import requests
import json

LATITUDE = 52.52  
LONGITUDE = 13.41  
BASE_URL = "https://api.open-meteo.com/"
ENDPOINT_1 = f"v1/forecast?latitude={LATITUDE}&longitude={LONGITUDE}&current_weather=true"

# Get-Request:
response = requests.get(BASE_URL + ENDPOINT_1)

# HTTP-Statuscode prüfen:
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4, sort_keys=True))
    current_weather = data["current_weather"]
    temperature = current_weather["temperature"]
    print(f"Temperature in Berlin: {temperature}")
    
else:
    print("Fehler beim Abrufen der Wetterdaten")
```

Es wäre jedoch angenehmer wenn wir eine Funktion hätten, welche wir nutzen könnten um die aktuelle Tempeartung von jeder Stadt abzurufen.
Dazu soll die Funktion Argumente erhalten:
- latitude
- longitude

```python
import requests
import json

def get_temperature(latitude: float, longitude: float, city_name: str = None):

    BASE_URL = "https://api.open-meteo.com/"
    ENDPOINT_1 = f"v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"

    response = requests.get(BASE_URL + ENDPOINT_1)

    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=4, sort_keys=True))
        current_weather = data["current_weather"]
        temperature = current_weather["temperature"]
        print(f"Temperatur in {city_name} ist: {temperature} Grad Celcius!")
        return temperature
        
    else:
        print("Fehler beim Abrufen der Wetterdaten")
        return None
    
if __name__ == "__main__":

    LATITUDE = 52.52  
    LONGITUDE = 13.41
    CITY_NAME = "Berlin"  

    berlin_temperature = get_temperature(LATITUDE, LONGITUDE, CITY_NAME)
```

Häufig ist es nicht angenehm die Ortsbezeichnung (z.B. Berlin) für die gegebenen Koordinaten (Latitude und Longitude) manuell herauszusuchen. Um dieses Problem zu umgehen können wir eine weitere API verwenden. Die API "OpenStreetMap" unter der Adresse https://nominatim.openstreetmap.org/ui/reverse.html? zu erreichen, ermöglicht uns aufgrundlage der Koordinaten die Ortsbezeichnung herauszufinden.

<img src="../img/requests_01.png" alt="Client-Server_Modell_01" width="450">

Die Dokumentation zur API findet man unter https://nominatim.org/release-docs/develop/api/Overview/<br>
Wir wollen eine Funktion erstellen, welche uns dies ermöglicht:

```python
def get_location_name(latitude: float, longitude: float) -> str:
    """Ermittelt nur den Stadtnamen aus den Koordinaten mit der OpenStreetMap Nominatim API."""
    
    BASE_URL = "https://nominatim.openstreetmap.org/"
    ENDPOINT_1 = f"reverse?lat={latitude}&lon={longitude}&format=json"
    url = f"{BASE_URL}{ENDPOINT_1}"  
    # Laut Dokumentation muss ein User-Agent angegeben werden:
    # https://operations.osmfoundation.org/policies/tiles/
    headers = {"User-Agent": "geo-request"}  
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=4, sort_keys=True))

        if "address" in data:
            address = data["address"]
            
            if "city" in address:
                return address["city"]
            elif "town" in address:
                return address["town"]
            elif "village" in address:
                return address["village"]
            elif "hamlet" in address:
                return address["hamlet"]
        return "Ort nicht gefunden"
    else:
        return "Fehler bei der Anfrage"
```

Wir haben in diesem Beispiel das erste mal einen User-Agent verwendet. Bildich kann man sich einen User-Agent als eine art "Ausweis" sich vorstellen, welcher bei einer Request mitgesendet wird. Mit dem User-Agent identifiziert man sich also bei dem Server, indem man mitteilt, wer man ist und woher die Request kommt. Der User-Agent wird in dem HTTP-Header angegeben wodurch die folgende Zeile zutande kommt:

```python
headers = {"User-Agent": "geo-request"} 
response = requests.get(url, headers=headers)
```

Der Server von OpenStreetMap würde ansonsten die Request einfach blockieren: https://operations.osmfoundation.org/policies/tiles/
<br>
<br>
Jetzt können wir unser Programm erweitern, indem wir beide Funktionen gleichzeitig verwenden. Dadurch können wir an die Funktion "get_temperature()" den Wert für das Argument "city_name" durch "get_location_name" übergeben:

```python
if __name__ == "__main__":

    LATITUDE = 52.52  
    LONGITUDE = 13.41 

    first_location = get_location_name(LATITUDE, LONGITUDE)
    berlin_temperature = get_temperature(LATITUDE, LONGITUDE, first_location)

```

Es wäre schön wenn man nicht nur einen Standort verwenden kann, sondern mehrere. Dabei sollen die Eingabedaten folgende Struktur aufweisen:

```python
locations = [
    {"latitude": 52.52, "longitude": 13.41},  # Berlin
    {"latitude": 48.85, "longitude": 2.35},   # Paris
    {"latitude": 40.71, "longitude": -74.01}, # New York
    {"latitude": 35.68, "longitude": 139.69}  # Tokio
]
```

Dies lässt sich leicht in der "get_temperature()" Funktion durch eine for-Schleife realisieren. So sieht nun das gesamte Programm aus:

```python
import requests
import json

def get_temperature(locations: list):
    """Holt die aktuelle Temperatur für eine Liste von Standorten mit Open-Meteo und speichert die Ergebnisse."""

    BASE_URL = "https://api.open-meteo.com/"
    temperature_storage = []

    for location in locations:
        latitude = location["latitude"]
        longitude = location["longitude"]
        location_name = get_location_name(latitude, longitude)

        ENDPOINT_1 = f"v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        response = requests.get(BASE_URL + ENDPOINT_1)

        if response.status_code == 200:
            data = response.json()
            current_weather = data["current_weather"]
            temperature = current_weather["temperature"]

            temperature_storage.append({
                "city": location_name,
                "latitude": latitude,
                "longitude": longitude,
                "temperature": temperature
            })
            print(f"Temperatur in {location_name}: {temperature}°C")
            
        else:
            print(f"Fehler beim Abrufen der Wetterdaten für {location_name}")
            temperature_storage.append({
                "city": location_name,
                "latitude": latitude,
                "longitude": longitude,
                "temperature": None
            })

    return temperature_storage
    
def get_location_name(latitude: float, longitude: float) -> str:
    """Ermittelt nur den Stadtnamen aus den Koordinaten mit der OpenStreetMap Nominatim API."""
    
    BASE_URL = "https://nominatim.openstreetmap.org/"
    ENDPOINT_1 = f"reverse?lat={latitude}&lon={longitude}&format=json"
    url = f"{BASE_URL}{ENDPOINT_1}"  
    # Laut Dokumentation muss ein User-Agent angegeben werden:
    headers = {"User-Agent": "geo-request"}  
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()

        if "address" in data:
            address = data["address"]
            
            if "city" in address:
                return address["city"]
            elif "town" in address:
                return address["town"]
            elif "village" in address:
                return address["village"]
            elif "hamlet" in address:
                return address["hamlet"]
        return "Ort nicht gefunden"
    else:
        return "Fehler bei der Anfrage"


if __name__ == "__main__":

    locations = [
        {"latitude": 52.52, "longitude": 13.41},  # Berlin
        {"latitude": 48.85, "longitude": 2.35},   # Paris
        {"latitude": 40.71, "longitude": -74.01}, # New York
        {"latitude": 35.68, "longitude": 139.69}  # Tokio
    ]

    temperature_data = get_temperature(locations)
    print(json.dumps(temperature_data, indent=4, sort_keys=True))
```

Das Programm funktionier soweit sehr gut, jedoch ist eine Auffälligkeit in der Ausgabe vorhanden:
- Es erscheinen japanische Zeichen anstelle "Tokio", weil die OpenStreetMap API den offiziellen Namen des Ortes in der Landessprache zurück gibt. Tokio ist in verschiedene Bezirke unterteilt, und „渋谷区“ (Shibuya-ku) ist einer davon. Jedoch sehen wir Unicode-Darstellung "\u6e0b\u8c37\u533a" des japanischen Textes. Mit Python können wir den Unicode dekodieren und den Inhalt lesbar machen.

Dazu müssen wir einfach einen Parameter der "dumps()" Funktion übergeben:

```python
  temperature_data = get_temperature(locations)
  print(json.dumps(temperature_data, indent=4, sort_keys=True, ensure_ascii=False))
```

Nun wollen wir unser Einführungsbeispiel mit einer sehr einfachen Datenbank erweitern, sodass Wetterdaten in einer SQLite-Datenbank gespeichert und später wieder abgerufen werden können. SQLite hat enige vorteille gegenüber anderen "schwergewichtigen" Datenbanken:
- SQLite ist eine leichte, dateibasierte Datenbank, die perfekt für kleine Projekte ist.
- Kein externer Server nötig – es funktioniert direkt mit einer lokalen Datei.
- SQL-Datenbanken sind besser strukturiert als JSON-Dateien oder csv-Dateien.

Auf der folgenden Seite findet man eine gute Dokumentation für einen schnellen Einsteig in die Datenbank Thematik: https://docs.python.org/3/library/sqlite3.html
Die SQLite Datenbank kann in Python über das Modul "sqlite3" verwendet werden. Statt also die Wetterdaten in einer JSON-Datei zu speichern, speichern wir sie strukturiert in einer Datenbank. Als erstes erstellen wir eine Funktion, welche eine Datenbank erstellt und eine Tabelle anlegt:

```python
def create_database():
    """Erstellt die SQLite-Datenbank und die Tabelle wetterdaten, falls sie noch nicht existiert."""

    # Verbindung zur Datenbank herstellen (erstellt Datei, falls nicht vorhanden):
    conn = sqlite3.connect("wetter.db")
    # Cursor erstellen (ein spezielles Objekt, das SQL-Befehle ausführt):
    cursor = conn.cursor()
    # Tabelle erstellen (falls nicht vorhanden), führt ein SQL-Statement aus:
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS wetterdaten (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            city TEXT,
            latitude REAL,
            longitude REAL,
            temperature REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
```

Unsere Datenbank Tabelle hat folgende Struktur:

| id              | city           | latitude   | longitude  | temperature | timestamp               |
| --------------- | -------------- | ---------- | ---------- | ----------- | ----------------------- |
| Automatische ID | Name der Stadt | Koordinate | Koordinate | Temperatur  | Automatische Zeitangabe |

Jetzt müssen wir einfach die Wetterdaten aus unserer API-Request in unserer Datenbank speichern. Dazu erstellen wir die folgende Funktion:

```python
def save_weather_data_to_db(city, latitude, longitude, temperature):
    """Speichert Wetterdaten in der SQLite-Datenbank wetter.db.""" 

    conn = sqlite3.connect("wetter.db")
    cursor = conn.cursor()

    # SQL-Befehl zum Einfügen von Daten:
    cursor.execute("""
        INSERT INTO wetterdaten (city, latitude, longitude, temperature) 
        VALUES (?, ?, ?, ?)
    """, (city, latitude, longitude, temperature))

    conn.commit() 
    conn.close() 
```

Hier nochmal eine kurze Wiederhollung zu SQL:
- "INSERT INTO wetterdaten (...) VALUES (?, ?, ?, ?)": Fügt neue Daten in die Tabelle ein
- "?" sind Platzhalter, die mit (city, latitude, longitude, temperature) befüllt werden

Jetzt brauchen wir noch eine Möglichkeit um die Wetterdaten aus unserer Datenbank lesen zu können. Dazu erstellen wir die folgende Funktion, welche alle gespeicherten Daten ausgibt:

```python
def get_saved_weather():
  """Ruft alle gespeicherten Wetterdaten aus der Datenbank ab."""

  conn = sqlite3.connect("wetter.db")
  cursor = conn.cursor()

  # Ausführen des SQL-Befehls:
  cursor.execute("SELECT * FROM wetterdaten ORDER BY timestamp DESC")
  data = cursor.fetchall()
  conn.close()

  return data
```

Folgendes bedeuten die SQL-Befehle bzw. sqlite-Methoden:
- "SELECT * FROM wetterdaten ORDER BY timestamp DESC": Holt alle Wetterdaten, sortiert nach neuesten zuerst.
- "cursor.fetchall()": Gibt eine Liste mit allen gespeicherten Daten zurück.